In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_csv('ChocolatePipes_trainData.csv')
df_label = pd.read_csv('ChocolatePipes_trainLabels.csv')


In [ ]:
# Sort dataframe based on id
df_train = df_train.sort_values('id')
df_label = df_label.sort_values('id')

n1,d1 = df_train.shape
n2,d2 = df_label.shape

# Drop unmatched intances and duplicates, step #1
label1 = df_train['id'].values.tolist()
label2 = df_label['id'].values.tolist()

labels = list(set(label1)-set(label2))
df_train = df_train[~df_train['id'].isin(labels)]
labels = list(set(label2)-set(label1))
df_label = df_label[~df_label['id'].isin(labels)]

df_label = df_label.drop_duplicates(subset = 'id', keep = 'first')

# Reset index
df_train.reset_index(drop = True, inplace = True)
df_label.reset_index(drop = True, inplace = True)

In [ ]:
# Merge into one dataframe
df = pd.concat([df_train, df_label['label']], axis = 1)
df

,id,Size of chocolate pool,Date of entry,Country funded by,Height of pipe,oompa loomper,longitude,Lattitude,Cocoa farm,Country of factory,...,Type of pump,management,management_group,Payment scheme,chocolate_quality,chocolate_quantity,chocolate_source,chocolate_source_class,pipe_type,label
0,0,0.0,11/13/13,63.0,0,60.0,233.125828,319.881846,5,2.0,...,0,0,user-group,6,2,0,5,0,2,0
1,1,0.0,3/5/12,72.0,2178,35.0,234.770717,315.604358,7,2343.0,...,12,0,user-group,0,0,0,5,0,2,2
2,2,0.0,3/27/12,130.0,0,123.0,236.115056,318.720732,6,217.0,...,7,0,user-group,4,0,1,3,0,1,2
3,3,210.0,6/3/14,58.0,1839,49.0,237.147432,321.812446,2,1477.0,...,3,4,user-group,4,0,0,0,0,0,2
4,4,0.0,3/22/12,401.0,0,417.0,236.164893,318.900711,6,1873.0,...,8,0,user-group,6,0,2,5,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47457,74240,0.0,3/22/14,12.0,1383,2.0,237.007726,321.719132,2,5354.0,...,3,5,user-group,6,0,1,1,1,0,2
47458,74242,0.0,4/12/12,11.0,0,9.0,233.724987,316.059242,7,328.0,...,3,1,user-group,5,0,0,6,1,0,2
47459,74243,0.0,11/13/13,89.0,1388,55.0,233.963539,323.570523,1,9832.0,...,14,0,user-group,0,0,0,8,1,1,0
47460,74246,250.0,3/7/12,60.0,1628,51.0,235.630481,317.289451,7,104.0,...,3,0,user-group,4,0,2,0,0,0,0


In [ ]:
# df = df.drop(['id'], axis = 1)   # Drop the Id field, step #2
df = df.drop(['Recorded by'], axis = 1)  # Remove single-value feature, step #2

# Drop categorical features that have low correlation and that are hard to encode, step #2
df = df.drop(['Country funded by'], axis = 1)
df = df.drop(['Country of factory'], axis = 1)
df = df.drop(['oompa loomper'], axis = 1)

# Find percentage of missing features
percent_missing_values = (df.isnull().sum(axis=0)/df.shape[0]).sort_values(ascending=False)
percent_missing_values = percent_missing_values[percent_missing_values > 0]
print(percent_missing_values)

# Impute missing values, step #3
df['Oompa loompa management'].fillna(df['Oompa loompa management'].mode()[0],inplace=True)
# df['oompa loomper'].fillna(df['oompa loomper'].mode()[0],inplace=True)
# df['Country funded by'].fillna(df['Country funded by'].mode()[0],inplace=True)
df['Does factory offer tours'].fillna(df['Does factory offer tours'].mode()[0],inplace=True)
df['Official or Unofficial pipe'].fillna(df['Official or Unofficial pipe'].mode()[0],inplace=True)
# df['Country of factory'].fillna(df['Country of factory'].mode()[0],inplace=True)

dfinfo = df.describe()

Oompa loompa management        0.065863
Does factory offer tours       0.055687
Official or Unofficial pipe    0.051094
dtype: float64


In [ ]:
print(df.corr().label)  # Find correlation with the label
print(df.columns)

# Modify date of entry, step #4
df['Date of entry'] = pd.to_datetime(df['Date of entry'])
df['Year of entry'] = df['Date of entry'].dt.year
df['Month of entry'] = df['Date of entry'].dt.month
df = df.drop(['Date of entry'], axis = 1)

# Remove outliers and abnormal zeros, step #5
outliercol = ['Size of chocolate pool']
zerocol = ['Size of chocolate pool', 'Year constructed', 'Height of pipe', 'Chocolate consumers in town']
n,d = df.shape
removelist = []
for i in range(n):
    for x in outliercol:
        mean = dfinfo.loc['mean'][x]
        std = dfinfo.loc['std'][x]
        if df.loc[i][x]-mean > 10*std:
            removelist.append(i)
    
    for y in zerocol:
        if df.loc[i][y] == 0 and ~(i in removelist):
            df.loc[i, y] = dfinfo.loc['75%'][y]
            
print(removelist)  

df = df.drop(removelist)        

id                            -0.006465
Size of chocolate pool         0.062836
Height of pipe                 0.113106
longitude                     -0.001786
Lattitude                      0.015100
Cocoa farm                    -0.011313
Region code                   -0.112987
District code                 -0.066833
Location                      -0.100549
Chocolate consumers in town    0.022614
Does factory offer tours       0.067222
Oompa loompa management        0.091728
Year constructed               0.045912
Type of pump                  -0.110630
management                     0.072328
Payment scheme                 0.076374
chocolate_quality             -0.111409
chocolate_quantity            -0.261659
chocolate_source              -0.094407
chocolate_source_class         0.027126
pipe_type                     -0.190013
label                          1.000000
Name: label, dtype: float64
Index(['id', 'Size of chocolate pool', 'Date of entry', 'Height of pipe',
       'longitude'

In [ ]:
# Encode binary features, step #6
df['Official or Unofficial pipe'] = df['Official or Unofficial pipe'].map({'Official pipe': 1, 'Unofficial pipe': 0})
df['Does factory offer tours'] = df['Does factory offer tours'].map({True: 1, False: 0})
print(df.corr().label)

# OHE categorical features, step #7
catecol = ['management_group', 'chocolate_source', 'chocolate_source_class', 
           'pipe_type', 'Payment scheme', 'Type of pump']
for col in catecol:
    dummies_train = pd.get_dummies(df[col], prefix = col)      # one hot encode all string valued columns
    df = pd.concat([df, dummies_train], axis = 1)

df = df.drop(catecol, axis = 1) 
df

id                            -0.006390
Size of chocolate pool         0.103460
Height of pipe                 0.112258
longitude                     -0.001880
Lattitude                      0.015348
Cocoa farm                    -0.011660
Region code                   -0.112837
District code                 -0.066702
Location                      -0.100337
Chocolate consumers in town    0.011555
Does factory offer tours       0.067485
Oompa loompa management        0.091282
Official or Unofficial pipe    0.036692
Year constructed               0.205327
Type of pump                  -0.110705
management                     0.071888
Payment scheme                 0.076340
chocolate_quality             -0.111295
chocolate_quantity            -0.261495
chocolate_source              -0.094626
chocolate_source_class         0.026827
pipe_type                     -0.189773
label                          1.000000
Year of entry                 -0.018846
Month of entry                -0.026325


,id,Size of chocolate pool,Height of pipe,longitude,Lattitude,Cocoa farm,Region code,District code,Location,Chocolate consumers in town,...,Type of pump_8,Type of pump_9,Type of pump_10,Type of pump_11,Type of pump_12,Type of pump_13,Type of pump_14,Type of pump_15,Type of pump_16,Type of pump_17
0,0,220.0,1520.0,233.125828,319.881846,5,339,4,107,1215.0,...,0,0,0,0,0,0,0,0,0,0
1,1,220.0,2178.0,234.770717,315.604358,7,336,5,15,1020.0,...,0,0,0,0,1,0,0,0,0,0
2,2,220.0,1520.0,236.115056,318.720732,6,326,5,23,1215.0,...,0,0,0,0,0,0,0,0,0,0
3,3,210.0,1839.0,237.147432,321.812446,2,328,6,63,1025.0,...,0,0,0,0,0,0,0,0,0,0
4,4,220.0,1520.0,236.164893,318.900711,6,326,5,23,1215.0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47457,74240,220.0,1383.0,237.007726,321.719132,2,327,8,41,1350.0,...,0,0,0,0,0,0,0,0,0,0
47458,74242,220.0,1520.0,233.724987,316.059242,7,337,3,79,1215.0,...,0,0,0,0,0,0,0,0,0,0
47459,74243,220.0,1388.0,233.963539,323.570523,1,345,7,114,1095.0,...,0,0,0,0,0,0,1,0,0,0
47460,74246,250.0,1628.0,235.630481,317.289451,7,336,2,53,1001.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Standardize data, step #8
dfinfo = df.describe()

standcol = ['Size of chocolate pool', 'Height of pipe', 'longitude', 'Lattitude',
       'Chocolate consumers in town']

for col in standcol:
    df[col] = (df[col] - dfinfo.loc['mean'][col]) / dfinfo.loc['std'][col]


In [ ]:
df.to_csv('processedData.csv', index=False)

In [ ]:
# Data processing for tests
testdf = pd.read_csv('ChocolatePipes_gradingTestData.csv')
testdf

,id,Size of chocolate pool,Height of pipe,longitude,Lattitude,Cocoa farm,Region code,District code,Location,Chocolate consumers in town,...,Type of pump_7,Type of pump_8,Type of pump_9,Type of pump_11,Type of pump_12,Type of pump_13,Type of pump_14,Type of pump_15,Type of pump_16,Type of pump_17
0,47224,0.350142,0.404441,0.065542,0.692541,5,341,2,30,0.399130,...,0,0,0,0,0,0,0,0,0,0
1,21217,-0.215232,0.383148,0.141349,-1.119154,0,337,3,79,-0.094853,...,0,0,0,0,0,0,0,0,0,0
2,40243,-0.215232,-2.156496,0.266325,-0.777804,7,385,54,108,-0.586538,...,1,0,0,0,0,0,0,0,0,0
3,260,-0.215232,0.383148,0.122810,1.042646,1,344,4,120,-0.094853,...,0,0,0,0,0,0,0,1,0,0
4,22148,-0.215232,0.315398,0.232170,0.827874,2,328,5,20,-0.586538,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5935,9426,-0.215232,2.340145,0.152690,-1.226907,0,336,4,31,-0.094853,...,0,0,0,0,0,0,0,0,0,0
5936,41487,-0.215232,0.383148,0.133876,0.644667,4,342,4,6,-0.094853,...,0,0,1,0,0,0,0,0,0,0
5937,35002,4.756739,0.377341,0.172743,-1.413191,0,336,6,0,-0.340696,...,0,0,0,0,0,0,0,0,0,0
5938,13410,-0.223547,0.971602,0.226798,0.854583,2,328,6,63,-0.462468,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
testdf = testdf.drop(['Recorded by'], axis = 1) 
testdf = testdf.drop(['Country funded by'], axis = 1)
testdf = testdf.drop(['Country of factory'], axis = 1)
testdf = testdf.drop(['oompa loomper'], axis = 1)

testdf['Oompa loompa management'].fillna(df['Oompa loompa management'].mode()[0],inplace=True)
testdf['Does factory offer tours'].fillna(df['Does factory offer tours'].mode()[0],inplace=True)
testdf['Official or Unofficial pipe'].fillna(df['Official or Unofficial pipe'].mode()[0],inplace=True)

percent_missing_values = (testdf.isnull().sum(axis=0)/testdf.shape[0]).sort_values(ascending=False)
percent_missing_values = percent_missing_values[percent_missing_values > 0]
print(percent_missing_values)

Series([], dtype: float64)


In [ ]:
zerocol = ['Size of chocolate pool', 'Year constructed', 'Height of pipe', 'Chocolate consumers in town']
n,d = testdf.shape

for i in range(n):   
    for x in zerocol:
        if testdf.loc[i][x] == 0:
            testdf.loc[i, x] = dfinfo.loc['75%'][x]

testdf['Date of entry'] = pd.to_datetime(testdf['Date of entry'])
testdf['Year of entry'] = testdf['Date of entry'].dt.year
testdf['Month of entry'] = testdf['Date of entry'].dt.month
testdf = testdf.drop(['Date of entry'], axis = 1)            

In [ ]:
testdf['Official or Unofficial pipe'] = testdf['Official or Unofficial pipe'].map({'Official pipe': 1, 'Unofficial pipe': 0})
testdf['Does factory offer tours'] = testdf['Does factory offer tours'].map({True: 1, False: 0})

catecol = ['management_group', 'chocolate_source', 'chocolate_source_class', 
           'pipe_type', 'Payment scheme', 'Type of pump']
for col in catecol:
    dummies_train = pd.get_dummies(testdf[col], prefix = col)      # one hot encode all string valued columns
    testdf = pd.concat([testdf, dummies_train], axis = 1)

testdf = testdf.drop(catecol, axis = 1) 

standcol = ['Size of chocolate pool', 'Height of pipe', 'longitude', 'Lattitude',
       'Chocolate consumers in town']

for col in standcol:
    testdf[col] = (testdf[col] - dfinfo.loc['mean'][col]) / dfinfo.loc['std'][col]

In [ ]:
testdf['Official or Unofficial pipe'].fillna(df['Official or Unofficial pipe'].mode()[0],inplace=True)
percent_missing_values = (testdf.isnull().sum(axis=0)/testdf.shape[0]).sort_values(ascending=False)
percent_missing_values = percent_missing_values[percent_missing_values > 0]
print(percent_missing_values)

testdf.insert(loc=43, column='pipe_type_6', value=np.zeros(5940))
testdf.insert(loc=61, column='Type of pump_10', value=np.zeros(5940))

Series([], dtype: float64)


In [ ]:
testdf.to_csv('ChocolatePipes_gradingTestData.csv', index=False)